# Import

In [45]:
import os
import re
import gc
import sys 
from datetime import datetime
from loguru import logger

from statsmodels.nonparametric.smoothers_lowess import lowess


import torch
import numpy as np
import pyqtgraph as pg
%gui qt



import matplotlib.pyplot as plt
%matplotlib qt




# Input Layer

## Definition

In [4]:
READ_RAW_FLAG = True

In [27]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list



def Load_Single(ele_id, rootPath, DATA_SUFFIX):
    fd_pt = os.path.join(f"{rootPath}/{ele_id}_归档", DATA_SUFFIX, f"{ele_id}_{DATA_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        logger.warning(f"{fd_pt} does not exist")
        return None
    data_pt = torch.load(fd_pt, weights_only=False)
    _data_group = data_pt["data_group"]

    return _data_group


def DRT_Lowess(DRTdata):
    '''==================================================
        DRT Analysis with Lowess
        Parameter: 
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
        Returen:
            DRTdata_Loess:    Loess smoothed DRT data
        ==================================================
    '''

    # _tau_i  = np.concatenate([i[0,1:-1] for i in DRTdata])
    # _R_i    = np.concatenate([i[1,1:-1] for i in DRTdata])
    # _C_i    = np.concatenate([i[2,1:-1] for i in DRTdata])
    _tau_i  = np.concatenate([i[0,:] for i in DRTdata])
    _R_i    = np.concatenate([i[1,:] for i in DRTdata])
    _C_i    = np.concatenate([i[2,:] for i in DRTdata])

    # _order  = _tau_i.argsort()
    # _tau_i  = _tau_i[_order]
    # _R_i    = _R_i[_order]
    # _C_i    = _C_i[_order]

    x_log = np.log(_tau_i)
    y_log = np.log(_R_i)-np.log(_C_i)
    
    y_log_smooth = lowess(y_log, x_log, frac=0.1, it=3, return_sorted=False)


    R_loess         = np.exp((x_log + y_log_smooth)/2)
    tau_loess       = _tau_i
    C_loess         = _tau_i / R_loess
    DRTdata_Loess   =  np.array([tau_loess, R_loess, C_loess])

    return DRTdata_Loess

def DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq):
    
    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # Text
    axis[4] = fig.add_subplot(2,3,5)    # DRT (RC)
    axis[5] = fig.add_subplot(2,3,6)    # DRT (Rτ)


    text_axis = axis[3]
    text_axis.axis('off')

    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(EISdata_list)):
        if i in eis_seq:
            ch_eis      = EISdata_list[i][0]
            ch_drt      = DRTdata_list[i]
            ch_loess    = Loess_list[i]

            # ch_R    = np.array([i[1:,0] for i in ch_drt])
            # ch_C    = np.array([i[1:,-1] for i in ch_drt])
            ch_R    = np.concatenate([i[1,:] for i in ch_drt])
            ch_C    = np.concatenate([i[2,:] for i in ch_drt])

            _color  = cmap(i/len(EISdata_list))

            axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
            axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

            axis[4].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[5].scatter(ch_loess[0,:], ch_loess[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')



    axis[0].set_aspect('equal', adjustable='datalim')
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[5].set_xscale('log')
    axis[5].set_yscale('log')


    return text_axis




## Archive Old - Loewner

### Extract All Data

In [6]:
if READ_RAW_FLAG:
    rootPath = "D:/Baihm/EISNN/Archive/"
    ele_list = SearchELE(rootPath, re.compile(r"(.+?)_归档"))
    DATASET_SUFFIX = "Outlier_Ver04"


    n_ele = len(ele_list)
    logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")


2025-07-24 12:33:11.968 | INFO     | __main__:<module>:8 - Search in D:/Baihm/EISNN/Archive/ and find 218 electrodes


In [84]:
if READ_RAW_FLAG:
    DRT_SUFFIX = f"{DATASET_SUFFIX}_DRTLoe_Ver02"

    vitro0_start_list = []
    vitro0_start_id_list = []
    vitro0_data_list = []
    vitro0_low_list = []
    vitro0_id_list = []

    n_avaliable = 0
    n_all_channel = 0

    for i in range(n_ele):
    # for i in range(3):
        fd_pt = os.path.join(ele_list[i][0], DRT_SUFFIX, f"{ele_list[i][1]}_{DRT_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue


        data_pt = torch.load(fd_pt)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]


        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        n_valid_ch  = len(_data_group["Channels"])

        x_day_full = _meta_group["TimeSpan"]
        _x_date = np.array([(poi - x_day_full[0]).days for poi in x_day_full])


        n_avaliable = n_avaliable + 1
        n_all_channel = n_all_channel + n_valid_ch

        
        logger.info(f"ELE [{i:03d}/{n_ele}][{n_valid_ch}-{n_all_channel}]: {ele_list[i][0]}")


        # Iteration by channel
        for j in _data_group['Channels']:
            try:
                _drt_data = _data_group[j]["DRTlist"]
                # _eis_data = _data_group[j]["EISlist"]
                _low_data = _data_group[j]["Loesslist"]

                eis_seq = _data_group[j]["eis_seq"]
                _drt_data = [_drt_data[ii] for ii in eis_seq]
                # _eis_data = [_eis_data[ii] for ii in eis_seq]
                _low_data = [_low_data[ii] for ii in eis_seq]

                _id_date = np.array(_x_date)
                _id_date = _id_date[eis_seq]

                vitro0_data_list.append(_drt_data)
                vitro0_start_list.append(_drt_data[0])

                vitro0_low_list.append(_low_data)

                _ch_id = j

                _id = [i, _ch_id] * len(_drt_data)
                _id = np.array(_id).reshape(-1,2)
                _eis_cluster = _data_group[j]['eis_cluster']
                _id = np.hstack((_id, _eis_cluster.reshape(-1,1)))
                _id = np.hstack((_id, _id_date.reshape(-1,1)))
                
                vitro0_id_list.append(_id)
                vitro0_start_id_list.append(_id[0,:])

            except Exception as e:
                logger.error(f"ELE[{ele_id}] - ch[{j:03d}] Run with error: {e}")
                continue




    vitro0_ele_list = [i[1] for i in ele_list]

    logger.info(f"Total {len(vitro0_data_list)} data points from {n_avaliable} electrodes")

    del data_pt, _meta_group, _data_group, _drt_data



C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_20504\4006158914.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
2025-07-24 15:21:44.966 | INF

In [74]:
len(vitro0_data_list[0][0][0])

3

### Plot Start Data

In [ ]:
# 创建窗口

app = pg.mkQApp()
win = pg.GraphicsLayoutWidget(show=True, title="Big Scatter Plot")
plot = win.addPlot()
plot.setWindowTitle('2e7 Points Scatter')
plot.setAspectLocked(False)

# 加载和flatten数据（你已有这步）
x_list = []
y_list = []

for i in range(len(vitro0_data_list)):
    for j in range(len(vitro0_start_list[i])):
    # for j in range(1):
        x = vitro0_start_list[i][j][1, :]
        y = vitro0_start_list[i][j][2, :]
        x_list.append(x)
        y_list.append(y)

x_all = np.concatenate(x_list).astype(np.float32)
y_all = np.concatenate(y_list).astype(np.float32)
# x_all = x_all[:10000]
# y_all = y_all[:10000]

scatter = pg.ScatterPlotItem(size=1, pen=None, brush=(255, 255, 255, 20))

# scatter.setData(x=np.log10(x_all), y=np.log10(y_all))
scatter.setData(x=np.log10(x_all)+np.log10(y_all), y=np.log10(x_all)-np.log10(y_all))

plot.addItem(scatter)
plot.setLogMode(x=True, y=True)


### Plot All Data

In [ ]:
# 创建窗口

app = pg.mkQApp()
win = pg.GraphicsLayoutWidget(show=True, title="Big Scatter Plot")
plot = win.addPlot()
plot.setWindowTitle('2e7 Points Scatter')
plot.setAspectLocked(False)

# 加载和flatten数据（你已有这步）
x_list = []
y_list = []

for i in range(len(vitro0_data_list)):
    for j in range(len(vitro0_data_list[i])):
        # for k in range(len(vitro0_data_list[i][j])):
        for k in range(20,22):
            x = vitro0_data_list[i][j][k][1, :]
            y = vitro0_data_list[i][j][k][2, :]
            x_list.append(x)
            y_list.append(y)

x_all = np.concatenate(x_list).astype(np.float32)
y_all = np.concatenate(y_list).astype(np.float32)
# x_all = x_all[:10000]
# y_all = y_all[:10000]

scatter = pg.ScatterPlotItem(size=1, pen=None, brush=(255, 255, 255, 20))

# scatter.setData(x=np.log10(x_all), y=np.log10(y_all))
# scatter.setData(x=np.log10(x_all)+np.log10(y_all), y=np.log10(x_all)-np.log10(y_all))

_tt = x_all * y_all
_yy = x_all / y_all
_yy = _yy / (((_tt/1e-7)**-1) + ((_tt/1e3)**1))
# _yy = (((_tt/1e-7)**-1) + ((_tt/1e3)**1))
scatter.setData(x=np.log10(_tt), y=np.log10(_yy))

plot.addItem(scatter)
plot.setLogMode(x=True, y=True)


In [128]:
import numpy as np
import matplotlib.pyplot as plt

x = np.logspace(-7, 1, 1000)
# y = (x/1e3)**-1 + (x/1e-7)**1
y = (x/1e-7)**-1 + (x/1e3)**1

plt.loglog(x, y)
plt.xlabel("x (log scale)")
plt.ylabel("y (log scale)")
plt.title("y = A*x^-α + B*x^β")
plt.grid(True)
plt.show()


### Extract Lowess

In [94]:
vitro0_lowess_list = []
# for i in range(len(vitro0_data_list)):
for i in range(20):
    for j in range(len(vitro0_data_list[i])):
        DRTdata_Loess = DRT_Lowess(vitro0_data_list[i][j])
        vitro0_lowess_list.append(DRTdata_Loess)
    if i%10 == 0:
        logger.info(f"Lowess [{i}/{len(vitro0_data_list)}]")

2025-07-24 15:58:46.821 | INFO     | __main__:<module>:8 - Lowess [0/12071]
2025-07-24 15:58:51.639 | INFO     | __main__:<module>:8 - Lowess [10/12071]


In [ ]:
# vitro0_lowess_align_list = []
# for i in range(len(vitro0_start_list)):
# # for i in range(3):
#     DRTdata_Loess = DRT_Lowess(vitro0_start_list[i])
#     vitro0_lowess_align_list.append(DRTdata_Loess)
#     if i%100 == 0:
#         logger.info(f"Lowess [{i}/{len(vitro0_start_list)}]")

2025-07-24 14:51:10.307 | INFO     | __main__:<module>:7 - Lowess [0/12071]


### Plot Lowess

In [100]:
import numpy as np
import pyqtgraph as pg
%gui qt
# 创建窗口

app = pg.mkQApp()
win = pg.GraphicsLayoutWidget(show=True, title="Big Scatter Plot")
plot = win.addPlot()
plot.setWindowTitle('2e7 Points Scatter')
plot.setAspectLocked(False)

# 加载和flatten数据（你已有这步）
x_list = []
y_list = []

for i in range(len(vitro0_lowess_list)):
        x = vitro0_lowess_list[i][1, :]
        y = vitro0_lowess_list[i][2, :]
        x_list.append(x)
        y_list.append(y)

x_all = np.concatenate(x_list).astype(np.float32)
y_all = np.concatenate(y_list).astype(np.float32)
# x_all = x_all[:10000]
# y_all = y_all[:10000]

scatter = pg.ScatterPlotItem(size=1, pen=None, brush=(255, 255, 255, 50))
# scatter.setData(x=np.log10(x_all), y=np.log10(y_all))
# scatter.setData(x=np.log10(x_all)+np.log10(y_all), y=np.log10(x_all)-np.log10(y_all))
scatter.setData(x=np.log10(x_all)+np.log10(y_all), y=np.log10(x_all))
# scatter.setData(x=np.log10(x_all)+np.log10(y_all), y=-np.log10(y_all))

plot.addItem(scatter)
plot.setLogMode(x=True, y=True)
# plot.setLogMode(x=True, y=False)


# Archive Old - Tik

## Extract All Data

In [41]:
if READ_RAW_FLAG:
    rootPath = "D:/Baihm/EISNN/Archive/"
    ele_list = SearchELE(rootPath, re.compile(r"(.+?)_归档"))
    DATASET_SUFFIX = "Outlier_Ver04"


    n_ele = len(ele_list)
    logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")


2025-07-24 13:43:39.767 | INFO     | __main__:<module>:8 - Search in D:/Baihm/EISNN/Archive/ and find 218 electrodes


In [44]:
if READ_RAW_FLAG:
    DRT_SUFFIX = f"{DATASET_SUFFIX}_DRTTik_Ver01"

    Loe_vitro0_start_list = []
    Loe_vitro0_start_id_list = []
    Loe_vitro0_data_list = []
    Loe_vitro0_id_list = []

    n_avaliable = 0
    n_all_channel = 0

    for i in range(n_ele):
    # for i in range(3):
        fd_pt = os.path.join(ele_list[i][0], DRT_SUFFIX, f"{ele_list[i][1]}_{DRT_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue


        data_pt = torch.load(fd_pt)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]


        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        n_valid_ch  = len(_data_group["Channels"])

        x_day_full = _meta_group["TimeSpan"]
        _x_date = np.array([(poi - x_day_full[0]).days for poi in x_day_full])


        n_avaliable = n_avaliable + 1
        n_all_channel = n_all_channel + n_valid_ch

        
        logger.info(f"ELE [{i:03d}/{n_ele}][{n_valid_ch}-{n_all_channel}]: {ele_list[i][0]}")


        # Iteration by channel
        for j in _data_group['Channels']:
            try:
                _drt_data = _data_group[j]["DRTdata"]
                _drt_data = [ii[[2,0,1],:] for ii in _drt_data]
                # _id_date = np.array(_x_date)
                # _id_date = _id_date[eis_seq]

                Loe_vitro0_data_list.append(_drt_data)
                Loe_vitro0_start_list.append(_drt_data[0])


                # _ch_id = j

                # _id = [i, _ch_id] * len(_drt_data)
                # _id = np.array(_id).reshape(-1,2)
                # _eis_cluster = _data_group[j]['eis_cluster']
                # _id = np.hstack((_id, _eis_cluster.reshape(-1,1)))
                # _id = np.hstack((_id, _id_date.reshape(-1,1)))
                
                # Loe_vitro0_id_list.append(_id)
                # Loe_vitro0_start_id_list.append(_id[0,:])

            except Exception as e:
                logger.error(f"ELE[{ele_id}] - ch[{j:03d}] Run with error: {e}")
                continue




    Loe_vitro0_ele_list = [i[1] for i in ele_list]

    logger.info(f"Total {len(Loe_vitro0_data_list)} data points from {n_avaliable} electrodes")

    del data_pt, _meta_group, _data_group, _drt_data



C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_20504\545717968.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
2025-07-24 14:02:26.606 | INFO

## Plot All Data

In [59]:


app = pg.mkQApp()
win = pg.GraphicsLayoutWidget(show=True, title="Big Line Plot")
plot = win.addPlot()
plot.setWindowTitle('2e7 Points Line')
plot.setAspectLocked(False)

# 加载和 flatten 数据（你已有这步）
x_list = []
y_list = []

for i in range(len(Loe_vitro0_start_list)):
    x = Loe_vitro0_start_list[i][1, :]
    y = Loe_vitro0_start_list[i][2, :]
    x_list.append(x)
    y_list.append(y)

x_all = np.concatenate(x_list).astype(np.float32)
y_all = np.concatenate(y_list).astype(np.float32)

# 做 log 变换（注意不加 setLogMode）
# x_log = np.log10(x_all)
# y_log = np.log10(y_all)

# x_log = np.log10(x_all) + np.log10(y_all)
# y_log = np.log10(x_all) - np.log10(y_all)
x_log = np.log10(x_all) + np.log10(y_all)
y_log = x_all/y_all

# 用 PlotDataItem 画线
line = pg.PlotDataItem(x=x_log, y=y_log, pen=pg.mkPen((255, 255, 255, 10), width=1))
plot.addItem(line)
